In [1]:
import nltk
import sys
from nltk.corpus import brown

In [2]:
#预处理词库
brown_tags_words=[]
for sent in brown.tagged_sents():
    #增加开头
#     print(sent)
    brown_tags_words.append(('start','start'))
    #tag省略前两个字母
    brown_tags_words.extend([(tag[:2],word)for (word,tag) in sent])
    #增加结尾
    brown_tags_words.append(('end','end'))

In [3]:
brown_tags_words[:10]
# len(brown_tags_words)

[('start', 'start'),
 ('AT', 'The'),
 ('NP', 'Fulton'),
 ('NN', 'County'),
 ('JJ', 'Grand'),
 ('NN', 'Jury'),
 ('VB', 'said'),
 ('NR', 'Friday'),
 ('AT', 'an'),
 ('NN', 'investigation')]

In [69]:
count_word_at=0
for elem in brown_tags_words:
#     print(elem[1])
    if elem[0]=='AT':
#         print(elem[1])
        count_word_at+=1
count_word_at


99077

In [4]:
from nltk.probability import ConditionalFreqDist
from nltk.probability import ConditionalProbDist,ELEProbDist
from nltk.tokenize import word_tokenize

In [5]:
#调用内置函数进行单统计P(wi | ti) = count(wi, ti) / count(ti)
cfd_tagwords=ConditionalFreqDist(brown_tags_words)
cpd_tagwords=nltk.ConditionalProbDist(cfd_tagwords,nltk.MLEProbDist)
print(cpd_tagwords)

<ConditionalProbDist with 51 conditions>


In [6]:
cpd_tagwords["VB"].prob("duck")

6.042713350943527e-05

In [7]:
# P(ti | t{i-1}) = count(t{i-1}, ti) / count(t{i-1})
brown_tags=[tag for (tag,word) in brown_tags_words]

In [8]:
#bigram:前后两个连在一起组成一组 count(t{i-1},ti)
cfd_tags=nltk.ConditionalFreqDist(nltk.bigrams(brown_tags))
#P（ti/t{i-1}）
cpd_tags=nltk.ConditionalProbDist(cfd_tags,nltk.MLEProbDist)

In [9]:
cpd_tags['DT'].prob('NN')

0.5057722522030194

In [10]:
distinct_tags=set(brown_tags)

In [11]:
sentence=['I','like','studying']
sentlen=len(sentence)

In [12]:
viterbi=[]

In [13]:
backpointer=[]

In [14]:
first_viterbi={}
first_backpointer={}
for tag in distinct_tags:
    if tag=='start':continue
    first_viterbi[tag]=cpd_tags['start'].prob(tag)*cpd_tagwords[tag].prob(sentence[0])
    first_backpointer[tag]='start'
print(first_viterbi)
print(first_backpointer)

{'(-': 0.0, 'IN': 0.0, ':': 0.0, 'HV': 0.0, 'AP': 0.0, '.-': 0.0, 'RB': 0.0, 'NI': 3.3324520848931064e-07, 'VB': 0.0, 'end': 0.0, '(': 0.0, '``': 0.0, 'RN': 0.0, ':-': 0.0, 'CC': 0.0, 'WR': 0.0, ')-': 0.0, 'MD': 0.0, 'NN': 1.0580313619573935e-06, '--': 0.0, 'NP': 1.7319067623793952e-06, 'FW': 0.0, ',': 0.0, 'NR': 0.0, '*-': 0.0, 'WD': 0.0, "''": 0.0, "'": 0.0, 'DT': 0.0, 'DO': 0.0, 'BE': 0.0, 'PP': 0.014930900689060006, 'WP': 0.0, 'PN': 0.0, 'EX': 0.0, 'UH': 0.0, 'JJ': 0.0, 'WQ': 0.0, 'CS': 0.0, '.': 0.0, 'RP': 0.0, 'TO': 0.0, 'AT': 0.0, 'CD': 0.0, ',-': 0.0, ')': 0.0, 'QL': 0.0, 'OD': 0.0, '*': 0.0, 'AB': 0.0}
{'(-': 'start', 'IN': 'start', ':': 'start', 'HV': 'start', 'AP': 'start', '.-': 'start', 'RB': 'start', 'NI': 'start', 'VB': 'start', 'end': 'start', '(': 'start', '``': 'start', 'RN': 'start', ':-': 'start', 'CC': 'start', 'WR': 'start', ')-': 'start', 'MD': 'start', 'NN': 'start', '--': 'start', 'NP': 'start', 'FW': 'start', ',': 'start', 'NR': 'start', '*-': 'start', 'WD': '

In [15]:
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

In [16]:
currbest=max(first_viterbi.keys(),key=lambda tag:first_viterbi[tag])
print('word','‘'+sentence[0]+'’','current best two_tag sequence',
     first_backpointer[currbest],currbest)

word ‘I’ current best two_tag sequence start PP


In [41]:
for wordindex in range(1,len(sentence)):
    this_viterbi={ }
    this_backpointer={ }
    prev_viterbi=viterbi[-1]
    
    for tag in distinct_tags:
        if tag=='True':continue
        #如果这个词的tag是X，现在的单词是w
        #我们想找前一个tag,并且让最好的tag sequence以Y,X结尾
        #也就是说，Y也要能最大化
        #prev_viterbi[Y]*P(X|Y)*P(w/X)
        best_previous=max(prev_viterbi.keys(),key=lambda prevtag:prev_viterbi[prevtag]*cpd_tags[prevtag].prob(tag)*cpd_tagwords[tag].prob(sentence[wordindex]))
        this_viterbi[tag]=prev_viterbi[best_previous]*cpd_tags[best_previous].prob(tag)*cpd_tagwords[tag].prob(sentence[wordindex])
        this_backpointer[tag]=best_previous
    currbest=max(this_viterbi.keys(),key=lambda tag:this_viterbi[tag])
    print('word','‘'+sentence[wordindex]+'‘','current best two_bag sequence',
         this_backpointer[currbest],currbest)
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)
#     print(viterbi)
#     print(backpointer)
                                                                                                            
                                                                                                               
                                                                                                               

[{'(-': '(-', 'IN': '(-', ':': '(-', 'HV': '(-', 'AP': '(-', '.-': '(-', 'RB': '(-', 'NI': '(-', 'VB': 'CS', 'end': '(-', '(': '(-', '``': '(-', 'RN': '(-', ':-': '(-', 'CC': '(-', 'WR': '(-', ')-': '(-', 'MD': '(-', 'NN': '(-', '--': '(-', 'NP': '(-', 'FW': '(-', ',': '(-', 'NR': '(-', '*-': '(-', 'WD': '(-', "''": '(-', "'": '(-', 'DT': '(-', 'DO': '(-', 'BE': '(-', 'PP': '(-', 'WP': '(-', 'PN': '(-', 'EX': '(-', 'UH': '(-', 'JJ': '(-', 'WQ': '(-', 'CS': '(-', '.': '(-', 'RP': '(-', 'TO': '(-', 'AT': '(-', 'CD': '(-', ',-': '(-', ')': '(-', 'start': '(-', 'QL': '(-', 'OD': '(-', '*': '(-', 'AB': '(-'}, {'(-': '(-', 'IN': 'PP', ':': '(-', 'HV': '(-', 'AP': '(-', '.-': '(-', 'RB': '(-', 'NI': '(-', 'VB': 'PP', 'end': '(-', '(': '(-', '``': '(-', 'RN': '(-', ':-': '(-', 'CC': '(-', 'WR': '(-', ')-': '(-', 'MD': '(-', 'NN': '(-', '--': '(-', 'NP': '(-', 'FW': '(-', ',': '(-', 'NR': '(-', '*-': '(-', 'WD': '(-', "''": '(-', "'": '(-', 'DT': '(-', 'DO': '(-', 'BE': '(-', 'PP': '(-', 'WP': 

In [38]:
#找所有以end结尾的tag sequence
prev_viterbi=viterbi[-1]
best_previous=max(prev_viterbi.keys(),key=lambda prevtag:
                 prev_viterbi[prevtag]*cpd_tags[prevtag].prob('end'))
prob_tagsequence=prev_viterbi[best_previous]*cpd_tags[best_previous].prob('end')
#倒着存，因此好的在后面
best_tagsequence=['end',best_previous]
#同理，这里也到过了
# backpointer.reverse()

In [35]:
#回溯所有的回点
current_best_tag=best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_bset_tag=bp[current_best_tag]

In [36]:
best_tagsequence.reverse()
print('the sentence was:',end=' ')
for w in sentence:
    print(w,end=' ')
print('\n')
print('best tag sequence is:',end=' ')
for t in best_tagsequence:
    print(t,end=' ')
print('\n')
print('the probability of the best tag sequence is:',prob_tagsequence)

the sentence was: I like studying 

best tag sequence is: start PP CS VB end 

the probability of the best tag sequence is: 6.223678544486739e-14
